In [1]:
# Basics
import os
import zipfile

# Data management
import pandas as pd

# Visualization
import matplotlib.pyplot as plt

# GIS/Maps
import geopandas as gpd
import folium

# For thematic map classification
import mapclassify


ModuleNotFoundError: No module named 'pandas'

In [ ]:
### Importing Streamlit necessities to deploy as an app
!pip install streamlit
!pip install pyngrok


In [ ]:
import os
import zipfile
import geopandas as gpd
import requests

# Step 1: Download the ZIP file from GitHub
url = "https://github.com/jlb719ru/researchassistantship2024/raw/main/County_Boundaries_(polygon)%20(3).zip"
response = requests.get(url)

# Check if the file was downloaded successfully
if response.status_code != 200:
    print(f"Error: File download failed. Status code: {response.status_code}")
else:
    print("File downloaded successfully!")

    # Step 2: Save the ZIP file to disk
    zip_file_path = '/content/County_Boundaries_(polygon) (3).zip'
    with open(zip_file_path, 'wb') as f:
        f.write(response.content)

    # Step 3: Extract the ZIP file
    extraction_path = '/content/shapefiles'
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extraction_path)

    # Step 4: List the extracted files
    extracted_files = os.listdir(extraction_path)
    print("Extracted files:", extracted_files)

    # Step 5: Find the shapefile (.shp) inside the extracted folder
    shapefile_path = next((os.path.join(extraction_path, f) for f in extracted_files if f.endswith('.shp')), None)

    if shapefile_path is None:
        print("Error: No shapefile (.shp) found in the extracted folder.")
    else:
        print(f"Shapefile found: {shapefile_path}")

        # Step 6: Read the shapefile using GeoPandas
        gdf = gpd.read_file(shapefile_path)

        # Step 7: Check the CRS of the shapefile
        print("Original CRS:", gdf.crs)

        # If the CRS is not EPSG:4326, reproject it
        if gdf.crs != 'EPSG:4326':
            gdf = gdf.to_crs(epsg=4326)  # Reproject to EPSG:4326
            print("Reprojected CRS:", gdf.crs)

        # Step 8: Filter the GeoDataFrame for counties in the Delaware Valley region
        delaware_valley_counties = [
            "Philadelphia", "Bucks", "Chester", "Delaware", "Montgomery",  # Pennsylvania counties
            "Burlington", "Camden", "Gloucester", "Mercer",  # New Jersey counties
            "New Castle",  # Delaware county
            "Cecil"  # Maryland county (optional)
        ]

        # Filter for the counties in the Delaware Valley region based on 'co_name' column
        delaware_valley_gdf = gdf[gdf['co_name'].isin(delaware_valley_counties)]

        # Display the first few rows of the filtered GeoDataFrame
        print("Counties in the Delaware Valley Region:")
        print(delaware_valley_gdf.head())

        # Optional: Save the filtered GeoDataFrame to a new shapefile or visualize
        # delaware_valley_gdf.to_file("/content/delaware_valley_counties.shp")
        # delaware_valley_gdf.plot()
        # plt.show()


File downloaded successfully!


7664898

Extracted files: ['County_Boundaries_(polygon).prj', 'County_Boundaries_(polygon).dbf', 'County_Boundaries_(polygon).cpg', 'County_Boundaries_(polygon).shp', 'County_Boundaries_(polygon).shx', 'County_Boundaries_(polygon).xml']
Shapefile found: /content/shapefiles/County_Boundaries_(polygon).shp
Original CRS: EPSG:26918
Reprojected CRS: EPSG:4326
Counties in the Delaware Valley Region:
    objectid  state_name state_fips cnty_fips   fips   area_sqmi state  \
1          2    Delaware         10       003  10003  481.162612    DE   
6          7    Maryland         24       015  24015  359.745162    MD   
16        17    Maryland         24       031  24031  507.162541    MD   
28        29  New Jersey         34       007  34007  227.423213    NJ   
32        33  New Jersey         34       015  34015  335.956240    NJ   

       co_name dvrpc_reg dvrpc_28co     co_seat dot_code    Shape__Are  \
1   New Castle        No        Yes  Wilmington     None  1.246210e+09   
6        Cecil    

In [ ]:
import folium

# Set the center of the map (you can change this based on your data)
map_center = [39.915, -75.2]  # Example center (near Philadelphia)

# Create a Folium map centered on the Delaware Valley region
m = folium.Map(location=map_center, zoom_start=8)

# Add the Delaware Valley region to the map as GeoJSON
folium.GeoJson(delaware_valley_gdf).add_to(m)

# Display the map
# m

In [ ]:
import folium
import pandas as pd
import time
from geopy.geocoders import Nominatim

# Initialize geolocator
geolocator = Nominatim(user_agent="school_locator")

# List of schools with optional addresses and school types
schools = [
    # Philadelphia County
    {"name": "Central High School", "type": "Charter Public", "address": "1700 W Olney Ave, Philadelphia, PA 19141"},
    {"name": "Northeast High School", "type": "Charter Public", "address": "1601 Cottman Ave, Philadelphia, PA 19111"},
    {"name": "Masterman School", "type": "Charter Public", "address": "1699 Spring Garden St, Philadelphia, PA 19130"},
    {"name": "Academy at Palumbo", "type": "Charter Public", "address": "1100 Catharine St, Philadelphia, PA 19147"},
    {"name": "George Washington High School", "type": "Charter Public", "address": "10175 Bustleton Ave, Philadelphia, PA 19116"},
    {"name": "Greenfield Elementary School", "type": "Charter Public", "address": "2200 Chestnut St, Philadelphia, PA 19103"},
    {"name": "MaST Community Charter Schools", "type": "Charter Public", "address": "1800 Byberry Rd, Philadelphia, PA 19116"},
    {"name": "KIPP Philadelphia Schools", "type": "Charter Public", "address": "2539 N 16th St, Philadelphia, PA 19132"},
    {"name": "Esperanza Academy Charter School", "type": "Charter Public", "address": "301 W Hunting Park Ave, Philadelphia, PA 19140"},
    {"name": "Germantown Friends School", "type": "Charter Public", "address": "31 W Coulter St, Philadelphia, PA 19144"},
    {"name": "Springside Chestnut Hill Academy", "type": "Charter Public", "address": "500 W Willow Grove Ave, Philadelphia, PA 19118"},
    {"name": "William Penn Charter School", "type": "Charter Public", "address": "3000 W School House Ln, Philadelphia, PA 19144"},
    {"name": "Roman Catholic High School", "type": "Charter Public", "address": "301 N Broad St, Philadelphia, PA 19107"},
    {"name": "St. Joseph's Preparatory School", "type": "Charter Public", "address": "1733 W Girard Ave, Philadelphia, PA 19130"},
    {"name": "Archbishop Ryan High School", "type": "Charter Public", "address": "11201 Academy Rd, Philadelphia, PA 19154"},

    # Bucks County
    {"name": "Pennsbury High School", "type": "Charter Public", "address": "705 Hood Blvd, Fairless Hills, PA 19030"},
    {"name": "Neshaminy High School", "type": "Charter Public", "address": "2001 Old Lincoln Hwy, Langhorne, PA 19047"},
    {"name": "Council Rock High School North", "type": "Charter Public", "address": "62 Swamp Rd, Newtown, PA 18940"},
    {"name": "Bucks County Montessori Charter School", "type": "Charter Public", "address": "1575 Edgewood Rd, Yardley, PA 19067"},
    {"name": "Solebury School", "type": "Charter Public", "address": "6832 Phillips Mill Rd, New Hope, PA 18938"},
    {"name": "Holy Ghost Preparatory School", "type": "Charter Public", "address": "2429 Bristol Pike, Bensalem, PA 19020"},
    {"name": "Villa Joseph Marie High School", "type": "Charter Public", "address": "1180 Holland Rd, Holland, PA 18966"},

    # Montgomery County
    {"name": "Lower Merion High School", "type": "Charter Public", "address": "315 E Montgomery Ave, Ardmore, PA 19003"},
    {"name": "North Penn High School", "type": "Charter Public", "address": "1340 S Valley Forge Rd, Lansdale, PA 19446"},
    {"name": "Wissahickon High School", "type": "Charter Public", "address": "521 Houston Rd, Ambler, PA 19002"},
    {"name": "Montgomery Flex Charter School", "type": "Charter Public", "address": "2340 Skippack Pike, Suite A, Blue Bell, PA 19422"},
    {"name": "The Haverford School", "type": "Charter Public", "address": "450 Lancaster Ave, Haverford, PA 19041"},
    {"name": "Baldwin School", "type": "Charter Public", "address": "701 Montgomery Ave, Bryn Mawr, PA 19010"},
    {"name": "Merion Mercy Academy", "type": "Charter Public", "address": "511 Montgomery Ave, Merion Station, PA 19066"},
    {"name": "Gwynedd Mercy Academy", "type": "Charter Public", "address": "1345 Sumneytown Pike, Gwynedd Valley, PA 19437"},

    # Camden County, New Jersey
    {"name": "Camden High School", "type": "Charter Public", "address": "1700 Park Blvd, Camden, NJ 08103"},
    {"name": "Cherry Hill High School East", "type": "Charter Public", "address": "1750 Kresson Rd, Cherry Hill, NJ 08003"},
    {"name": "LEAP Academy University Charter School", "type": "Charter Public", "address": "3500 Federal St, Camden, NJ 08105"},
    {"name": "Bishop Eustace Preparatory School", "type": "Charter Public", "address": "555 New Jersey 70, Pennsauken, NJ 08109"},
    {"name": "Our Lady of Mount Carmel Regional School", "type": "Charter Public", "address": "2301 S 9th St, Camden, NJ 08104"},

    # Burlington County, New Jersey
    {"name": "Lenape High School", "type": "Charter Public", "address": "235 W Clinton Ave, Medford, NJ 08055"},
    {"name": "Rancocas Valley Regional High School", "type": "Charter Public", "address": "520 Jacksonville Rd, Mount Holly, NJ 08060"},
    {"name": "New Jersey Virtual Charter School", "type": "Charter Public", "address": "2100 Springdale Rd, Cherry Hill, NJ 08003"},
    {"name": "Doane Academy", "type": "Charter Public", "address": "350 Riverbank Rd, Burlington, NJ 08016"},
    {"name": "Holy Cross Preparatory Academy", "type": "Charter Public", "address": "5035 Route 130, Delran, NJ 08075"},

    # New Castle County, Delaware
    {"name": "Newark High School", "type": "Charter Public", "address": "750 E Delaware Ave, Newark, DE 19711"},
    {"name": "Alexis I. du Pont High School", "type": "Charter Public", "address": "1301 Carruthers Ln, Wilmington, DE 19803"},
    {"name": "Charter School of Wilmington", "type": "Charter Public", "address": "100 N Dupont Rd, Wilmington, DE 19807"},
    {"name": "Delaware Military Academy", "type": "Charter Public", "address": "112 Middleboro Rd, Wilmington, DE 19804"},
    {"name": "Sanford School", "type": "Charter Public", "address": "6900 Lancaster Pike, Hockessin, DE 19707"},
    {"name": "Tatnall School", "type": "Charter Public", "address": "1501 Barley Mill Rd, Wilmington, DE 19807"},
    {"name": "Ursuline Academy", "type": "Charter Public", "address": "1106 Pennsylvania Ave, Wilmington, DE 19806"},
    {"name": "St. Mark's High School", "type": "Charter Public", "address": "2501 Pike Creek Rd, Wilmington, DE 19808"}
]


# Function to get latitude and longitude
def get_coordinates(name, address):
    try:
        location = geolocator.geocode(address)
        if location:
            return location.latitude, location.longitude
        else:
            print(f"Could not geocode: {name}")
            return None, None
    except Exception as e:
        print(f"Error for {name}: {e}")
        return None, None

# Dataframe to store results
results = []

# Loop through schools
for school in schools:
    name = school["name"]
    address = school["address"]
    lat, lon = get_coordinates(name, address)
    results.append({"name": name, "type": school["type"], "address": address, "latitude": lat, "longitude": lon})
    time.sleep(1)  # Pause to respect API usage limits

# Convert results to a DataFrame
school_coords = pd.DataFrame(results)

# Define colors for each school type
type_colors = {
    "Public District": "blue",
    "Charter": "green",
    "Private Secular": "purple",
    "Private Religious": "red"
}

# Plot each school on the map
for _, row in school_coords.iterrows():
    if not pd.isna(row["latitude"]) and not pd.isna(row["longitude"]):
        folium.Marker(
            location=[row["latitude"], row["longitude"]],
            popup=f"{row['name']} ({row['type']})",
            icon=folium.Icon(color=type_colors.get(row["type"], "gray"))
        ).add_to(m)

# Save or display the map
# m.save("schools_map.html")
m  # This will render the map if you are in a Jupyter Notebook environment.


Could not geocode: Bucks County Montessori Charter School
Could not geocode: Solebury School
Could not geocode: Montgomery Flex Charter School
Could not geocode: Bishop Eustace Preparatory School
Could not geocode: Lenape High School
Could not geocode: Holy Cross Preparatory Academy


KeyboardInterrupt: 